In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import scipy as scipy
import time
import pandas as pd
from sklearn.linear_model import LinearRegression
import re
import Betas as b
import multiprocessing
import validators 
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from itertools import combinations

In [2]:
UNCODES = '/Users/benjaminsionit/Downloads/UNCODES.csv'
UN = pd.read_csv(UNCODES)

DATA = '/Users/benjaminsionit/Downloads/FINALDATA.csv'
FINALDATA = pd.read_csv(DATA)


FINALDATAWITHREGIONS = pd.merge(UN,FINALDATA, right_on = list(FINALDATA.columns)[2], left_on = list(UN.columns)[2])
#print(list(FINALDATAWITHREGIONS.columns))
#FINALDATAWITHREGIONS

In [3]:
def SortBySubRegion(subregion):
    REGION = FINALDATAWITHREGIONS[FINALDATAWITHREGIONS['sub-region'] == subregion]
    if(sum(REGION['Year'] == 'NaN') == 0):
        return(REGION)
    else:
        print("There are NAs for the Year for the inputed sub-region")


def MeanforX(DF, colname):
    ForX = pd.DataFrame()
    for i in pd.unique(DF['Year']):
            Mean = pd.DataFrame({'Year': [i], colname: [np.mean(DF[DF['Year'] == i][colname])]})
            ForX = pd.concat([ForX, Mean])
    return(ForX)

In [4]:
COMPILEDDATA = pd.DataFrame()
for i in pd.unique(FINALDATAWITHREGIONS['sub-region']):
    SUBREGION = SortBySubRegion(i)
    SUBREGIONDATA = pd.DataFrame(np.arange(1992,2018+1))
    SUBREGIONDATA.columns = ['Year']
    for j in SUBREGION.columns[15:]:
        SUBREGIONDATA = pd.merge(SUBREGIONDATA,MeanforX(DF = SUBREGION,colname = j))
    COMPILEDDATA = pd.concat([COMPILEDDATA, SUBREGIONDATA])

In [5]:
COMPILEDDATA.to_csv(r'/Users/benjaminsionit/Downloads/COMPILEDDATA.csv', index=False, header=True)
FINALDATAWITHREGIONS.to_csv(r'/Users/benjaminsionit/Downloads/FINALDATAWITHREGIONS.csv', index=False, header=True)

In [6]:
print(pd.unique(FINALDATAWITHREGIONS['sub-region']))

['Southern Europe' 'Northern Africa' 'Sub-Saharan Africa'
 'Latin America and the Caribbean' 'Australia and New Zealand'
 'Western Europe' 'Southern Asia' 'South-eastern Asia' 'Eastern Europe'
 'Northern America' 'Polynesia' 'Western Asia' 'Northern Europe'
 'Eastern Asia' 'Micronesia' 'Melanesia' 'Central Asia']


In [7]:
NewFinalDataUrl  = '/Users/benjaminsionit/Downloads/NewDataFinal.csv'
final = pd.read_csv(NewFinalDataUrl)
final = final.drop(columns=['Unnamed: 0'])

y = final["Surface Temperature Change"]

listvals = list(final.columns[16:])
for i in range(len(listvals)):
    final[final.columns[16+i]] = (i+1)*final[final.columns[16+i]]
    
regionfactors = np.asmatrix(final[final.columns[16:]]).sum(axis=1)
final = final.drop(columns = final.columns[16:])
final['Region Factors'] = regionfactors

final

,const,Year,Land Coverage,Surface Temperature Change,BC,CO,NH3,NMVOC,NOx,OC,PM10,PM2.5,SO2,CO2,CH4,N2O,Region Factors
0,1.0,1992,69.264629,-0.579,0.853328,25.791130,3.039674,86.290665,16.088104,0.593082,2.143882,1.854832,2.794095,62683.453024,974.869899,2.596461,6
1,1.0,1992,84.831480,0.511,0.001142,0.583143,0.263843,0.094454,0.281613,0.000961,0.009401,0.004620,0.282423,476.083470,2.834219,0.061190,3
2,1.0,1992,76.205103,0.360,0.000121,0.058826,0.000058,0.006139,0.011386,0.000081,0.000237,0.000236,0.001254,7.902936,0.093497,0.000673,3
3,1.0,1992,95.192667,0.105,0.039752,5.993535,26.216038,4.831343,2.508253,0.228474,0.727263,0.505179,0.522469,4027.377643,103.980349,2.924359,2
4,1.0,1992,85.735300,0.606,89.150961,15859.028027,253.204942,320.788063,438.673385,802.504497,2215.204432,1549.418839,69.100699,27801.150922,645.354421,7.937473,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47819,1.0,2018,27.909300,0.405,3.355815,368.704732,61.522615,54.649509,19.284819,13.908446,52.477207,30.016152,11.528146,46425.903295,508.559022,16.531770,8
47820,1.0,2018,18.337500,0.405,3.355815,368.704732,61.522615,54.649509,19.284819,13.908446,52.477207,30.016152,11.528146,46425.903295,508.559022,16.531770,8
47821,1.0,2018,90.625800,0.405,3.355815,368.704732,61.522615,54.649509,19.284819,13.908446,52.477207,30.016152,11.528146,46425.903295,508.559022,16.531770,8
47822,1.0,2018,11645.811900,0.405,3.355815,368.704732,61.522615,54.649509,19.284819,13.908446,52.477207,30.016152,11.528146,46425.903295,508.559022,16.531770,8


In [8]:
X_variables = final.iloc[:, [0,1,2,4,5,6,7,8,9,10,11,12,13,14,15,16]]

vif_data = pd.DataFrame()
vif_data["feature"] = X_variables.columns
vif_data["VIF"] = [variance_inflation_factor(X_variables.values, i) for i in range(len(X_variables.columns))]


while max(vif_data['VIF']) > 10:
    X_variables = X_variables.drop(columns = [vif_data['feature'][vif_data.index[vif_data['VIF'].idxmax()]]])
    vif_data = pd.DataFrame()
    vif_data["feature"] = X_variables.columns
    vif_data["VIF"] = [variance_inflation_factor(X_variables.values, i) for i in range(len(X_variables.columns))]
    


vif_data

,feature,VIF
0,Year,5.370134
1,Land Coverage,1.125951
2,BC,2.285293
3,CO,1.789911
4,CO2,1.462585
5,Region Factors,5.289822


In [9]:
def model_fit_calculator(y, x):
    model = sm.OLS(y, x).fit()
    sse = np.sum((model.fittedvalues - y)**2)
    ssr = np.sum((model.fittedvalues - y.mean())**2)
    sst = ssr + sse
    AIC = len(y)*np.log(sse)-np.log(len(y))+2*x.shape[1]
    BIC = len(y)*np.log(sse)-np.log(len(y)) + (x.shape[1]*np.log(len(y)))
    R_squared = 1 - (ssr/sst)
    VIF = 1 - R_squared
    return(pd.Series({"Dependent": y.name, "Independent": list(x.columns)[0:], 
                      "AIC": AIC, "BIC": BIC, "R-Squared": R_squared, "VIF": VIF}))



In [10]:
model_info = pd.DataFrame(columns = ["Dependent", "Independent", "AIC", "BIC", "R-Squared", "VIF"])

for k in range(2,7):
    comb = list(combinations([0, 1, 2, 3, 4, 5], k))
    for j in range(len(comb)):
        model = X_variables.iloc[:, list(comb[j])]
        model_info = pd.concat([model_info, model_fit_calculator(y, model).to_frame().T], ignore_index = True)





In [11]:
model_info.sort_values(by = ["AIC"], ascending=True)
model_info.sort_values(by = ["BIC"], ascending=True)

,Dependent,Independent,AIC,BIC,R-Squared,VIF
56,Surface Temperature Change,"[Year, Land Coverage, BC, CO, CO2, Region Fact...",453731.773408,453784.425106,0.981902,0.018098
52,Surface Temperature Change,"[Year, Land Coverage, BC, CO2, Region Factors]",453743.7813,453787.657715,0.982197,0.017803
54,Surface Temperature Change,"[Year, BC, CO, CO2, Region Factors]",453764.694505,453808.57092,0.982617,0.017383
43,Surface Temperature Change,"[Year, BC, CO2, Region Factors]",453776.960481,453812.061613,0.982917,0.017083
53,Surface Temperature Change,"[Year, Land Coverage, CO, CO2, Region Factors]",453778.917042,453822.793456,0.982899,0.017101
51,Surface Temperature Change,"[Year, Land Coverage, BC, CO, Region Factors]",453788.982424,453832.858838,0.983124,0.016876
39,Surface Temperature Change,"[Year, Land Coverage, CO, Region Factors]",453805.708777,453840.809909,0.983499,0.016501
37,Surface Temperature Change,"[Year, Land Coverage, BC, Region Factors]",453806.631918,453841.73305,0.983537,0.016463
44,Surface Temperature Change,"[Year, CO, CO2, Region Factors]",453807.214797,453842.315929,0.98352,0.01648
23,Surface Temperature Change,"[Year, CO, Region Factors]",453855.580941,453881.90679,0.984567,0.015433


In [12]:
best_model = np.asmatrix(X_variables.iloc[:, [0,1,2,3,4,5]])
a = best_model.T @ best_model
print(np.linalg.matrix_rank(a) == a.shape[1])
y = np.asmatrix(y).T

True


In [13]:
# SVD decomp
start = time.time()
u, s, vh = np.linalg.svd(a,full_matrices=False)
smat = np.diag(s)
# Calculating beta
smatinv = np.diag((1/(s)))
uinv = np.linalg.inv(u)
vhinv = np.linalg.inv(vh)
beta = (vhinv @ smatinv @ uinv) @ best_model.T @ y
K = y - (best_model)@beta
var = (K.T @ K)/(best_model.shape[0] - best_model.shape[1]) # Calculating variance
std_error = var * np.diag(vhinv @ smatinv @ uinv) # Calculating standard error
end = time.time()
print(end - start, "\n\n", beta, "\n\n", std_error)

0.002672910690307617 

 [[ 4.77087986e-04]
 [ 4.36798945e-07]
 [-1.14997426e-03]
 [-2.30125223e-06]
 [ 3.62078905e-08]
 [-2.43129151e-02]] 

 [[7.70678292e-12 5.46224534e-15 2.68992586e-08 3.78047652e-13
  2.21311625e-17 1.03883595e-06]]


In [14]:
list(combinations([0, 1, 2, 3, 4, 5], 6))

[(0, 1, 2, 3, 4, 5)]